## [github](https://www.kaggle.com/kimjihoo/coronavirusdataset)에서 가져온 데이터 전처리

In [ ]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
import pandas as pd

### PatientRoute.csv

In [3]:
patient_route = pd.read_csv(os.path.join(root, 'data', 'PatientRoute.csv'))
patient_route.head()

,patient_id,global_num,date,province,city,type,latitude,longitude
0,1000000001,2.0,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534


### PatientInfo.csv

In [4]:
patient_info = pd.read_csv(os.path.join(root, 'data', 'PatientInfo.csv'))
patient_info.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released


In [5]:
patient_info = patient_info[['patient_id', 'global_num', 'sex', 'age', 'infection_case']]
patient_info.head()

,patient_id,global_num,sex,age,infection_case
0,1000000001,2.0,male,50s,overseas inflow
1,1000000002,5.0,male,30s,overseas inflow
2,1000000003,6.0,male,50s,contact with patient
3,1000000004,7.0,male,20s,overseas inflow
4,1000000005,9.0,female,20s,contact with patient


### MergedRoute.csv

In [6]:
merged_route = pd.merge(patient_info, patient_route)
merged_route.head()

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude
0,1000000001,2.0,male,50s,overseas inflow,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534


In [7]:
# sex, age 없는 경우 삭제
merged_route = merged_route[pd.notnull(merged_route['sex'])]
merged_route = merged_route[pd.notnull(merged_route['age'])]
merged_route.head()

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude
0,1000000001,2.0,male,50s,overseas inflow,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534


In [8]:
# latitude, longitude → grid 정보
def to_grid(lat, lon):
    row = 256
    column = 256
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.472494
    seoul_end_lon = 127.142928
    lat_distance = seoul_end_lat - seoul_start_lat 
    lon_distance = seoul_end_lon - seoul_start_lon

    if seoul_end_lat > lat or lat > seoul_start_lat:
        return -1, -1
    if seoul_start_lon > lon or lon > seoul_end_lon:
        return -1, -1

    lat_distance2= lat - seoul_start_lat
    lon_distance2 = lon - seoul_start_lon
    row = (lat_distance2 * 255) / lat_distance
    col = (lon_distance2 * 255) / lon_distance

    return int(row), int(col)

merged_route['row'] = ""
merged_route['col'] = ""

for i in range(len(merged_route)):
    latitude = merged_route['latitude'].values[i]
    longitude = merged_route['longitude'].values[i]
    row, col = to_grid(latitude, longitude)
    merged_route.iloc[i, merged_route.columns.get_loc('row')] = row
    merged_route.iloc[i, merged_route.columns.get_loc('col')] = col
    
merged_route.head()

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude,row,col
0,1000000001,2.0,male,50s,overseas inflow,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632,-1,-1
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659,147,152
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048,119,161
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822,120,161
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534,151,170


In [9]:
# 집단감염 value 수정
causes = merged_route['infection_case']
causes = causes.drop_duplicates(keep='last')
causes = causes.dropna()
causes = causes.tolist()
causes_stay = ["contact with patient", "overseas inflow", "etc"]

for i in range(len(merged_route)):
    cause = merged_route.iloc[i, merged_route.columns.get_loc('infection_case')]
    if cause in causes_stay: continue
    merged_route.iloc[i, merged_route.columns.get_loc('infection_case')] = "community infection"
    
merged_route.head()

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude,row,col
0,1000000001,2.0,male,50s,overseas inflow,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632,-1,-1
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659,147,152
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048,119,161
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822,120,161
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534,151,170


In [10]:
# 서울 아닌 확진자 방문지 제거, 필요 없는 columns 제거
merged_route = merged_route.loc[merged_route['row'] != -1]
merged_route = merged_route[['patient_id', 'sex', 'age', 'infection_case', 'date', 'type', 'latitude', 'longitude', 'row', 'col']]
merged_route.head()

,patient_id,sex,age,infection_case,date,type,latitude,longitude,row,col
1,1000000001,male,50s,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147,152
2,1000000002,male,30s,overseas inflow,2020-01-25,etc,37.592560,127.017048,119,161
3,1000000002,male,30s,overseas inflow,2020-01-26,store,37.591810,127.016822,120,161
4,1000000002,male,30s,overseas inflow,2020-01-26,public_transportation,37.563992,127.029534,151,170
5,1000000002,male,30s,overseas inflow,2020-01-26,public_transportation,37.590330,127.015221,121,160


In [ ]:
# 파일로 저장
merged_route.to_csv(os.path.join(root, 'data', 'MergedRoute.csv'), mode='w')